In [2]:
import numpy as np
import pandas as pd
import data as dt
import functions as fn
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as pta

In [4]:
BTCUSDT15m = dt.BTCUSDT15m

In [3]:
n = 14
n2 = 17

BTCUSDT15m['RSI_1'] = pta.rsi(BTCUSDT15m.Close, length = 14)

def Stoch_RSI(df,n,rsi_name):
    Stoch_RSI_t = []
    for i in range(len(df[rsi_name])-n):
        temp = df[rsi_name][i:i+n]
        L_RSI = np.min(temp)
        M_RSI = np.max(temp)
        RSI = temp.values[-1]

        Stoch_RSI_t.append(( RSI - L_RSI )   /  ( M_RSI - L_RSI))


    Stoch_RSI =  np.zeros(len(df))
    Stoch_RSI[n:] = np.array(Stoch_RSI_t)*100

    return Stoch_RSI

BTCUSDT15m['S_RSI_1'] = Stoch_RSI(BTCUSDT15m,n,'RSI_1')
BTCUSDT15m['D'] = BTCUSDT15m['S_RSI_1'].rolling(3).mean()

C:\Users\tonat\AppData\Local\Temp/ipykernel_77320/2793740361.py:14: RuntimeWarning: invalid value encountered in double_scalars
  Stoch_RSI_t.append(( RSI - L_RSI )   /  ( M_RSI - L_RSI))


In [4]:

BTCUSDT15m.set_index("Open Time", inplace=True)
t1 = BTCUSDT15m['2021-03-18 19:00:00': '2021-03-21 00:00:00']

In [5]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=t1.index,
            open=t1["Open"],
            high=t1["High"],
            low=t1["Low"],
            close=t1["Close"],
            name="OHLC",
        ),
        go.Scatter(
            x=t1.index, y=t1['S_RSI_1'], mode="lines", name="Stoch RSI", yaxis="y2"
        ),
        go.Scatter(
            x=t1.index, y=t1['D'], mode="lines", name="D%", yaxis="y2"
        ),
    ],
 ).update_layout(
     yaxis_domain=[0.3, 1],
     yaxis2={"domain": [0, 0.20]},
     xaxis_rangeslider_visible=False,
     showlegend=False,
 )

fig_001

### <font color = 'Green'>RSI, EMA200 & ATR Strategy</font>

In [13]:
# download the data
BTCUSDT15m = dt.BTCUSDT15m

# get the ema signal 
EMA = fn.EMA200_Signal(BTCUSDT15m)
BTCUSDT15m['EMA_Signal'] = EMA['EMA200_Signal']

# get the rsi
BTCUSDT15m['RSI'] = pta.rsi(BTCUSDT15m.Close, length = 14)

# get ATR
BTCUSDT15m['ATR'] = pta.atr(BTCUSDT15m['High'],BTCUSDT15m['Low'],BTCUSDT15m['Close'])

In [7]:
BTCUSDT15m['TotSignal'] = fn.TotSignal(BTCUSDT15m)

BTCUSDT15m.dropna(inplace=True)

In [8]:
BTCUSDT15m['pointpos'] = BTCUSDT15m.apply(lambda row: fn.pointpos(row), axis=1)
BTCUSDT15m.reset_index(drop = True, inplace=True)

In [9]:
months = 24*4*30
startid = 7000

dfpl = BTCUSDT15m[startid:startid+int(.4*months)]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA200, line=dict(color='red', width=1.5), name="EMA200")])

fig.add_scatter(x = dfpl.index, y = dfpl['pointpos'], mode = 'markers', 
                marker = dict(size = 5, color = 'MediumPurple'), name = 'signal')                
fig.show()

### Back testing strategy

In [10]:
#! pip install backtesting
from backtesting import Strategy
from backtesting import Backtest

In [19]:
months = 24*4*30
startid = 0
BTCUSDT15m.set_index("Open Time", inplace=True)
dfpl = BTCUSDT15m#[startid:startid+1*months]


def SIGNAL():
    return dfpl.TotSignal

In [27]:
class MyStrat(Strategy):
    initsize = .2
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = self.data.ATR[-1]
        TPSLRatio = 1.5

        if self.signal1 == 2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1,size=self.mysize)
        
        elif self.signal1 == 1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl = sl1, tp=tp1,size=self.mysize)


bt = Backtest(dfpl, MyStrat, cash=1000000, commission=.00)
stat = bt.run()
stat

Start                     2018-01-03 01:45:00
End                       2022-07-06 03:45:00
Duration                   1645 days 02:00:00
Exposure Time [%]                    0.445351
Equity Final [$]               1049603.051887
Equity Peak [$]                1052151.889461
Return [%]                           4.960305
Buy & Hold Return [%]               34.739001
Return (Ann.) [%]                    0.969647
Volatility (Ann.) [%]                1.014272
Sharpe Ratio                         0.956003
Sortino Ratio                        1.687456
Calmar Ratio                         0.830726
Max. Drawdown [%]                   -1.167228
Avg. Drawdown [%]                   -0.222399
Max. Drawdown Duration      459 days 12:30:00
Avg. Drawdown Duration       42 days 14:03:00
# Trades                                  169
Win Rate [%]                        46.745562
Best Trade [%]                        2.82857
Worst Trade [%]                     -1.716045
Avg. Trade [%]                    